# Q&A against a SQL Database (Azure SQL, Azure Fabric, Synapse, SQL Managed Instance, etc)

CSV 파일에 대한 표 형식의 데이터를 쿼리하는 방법을 이전 노트북에서 배웠으므로 이제 데이터의 원본을 기준으로 SQL 데이터베이스에 직접 질문을 해보겠습니다.
이 노트북의 목표는 GPT-4와 같이 뛰어난 LLM이 어떻게 인간의 질문을 이해하고 이를 SQL 쿼리로 변환하여 답을 얻을 수 있는지 보여줍니다.

우리는 처음 배포한 환경의 Azure SQL Server를 사용할 것입니다. 그러나 아래와 같은 코드는 Synapse와 같은 SQL Database에서 작동합니다. 서버는 Azure Cognitive Search 서비스가 위치한 리소스 그룹에 생성되어야 합니다.

시작해보겠습니다.

In [2]:
import os
import pandas as pd
import pyodbc
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor
from langchain.callbacks.manager import CallbackManager

from common.prompts import MSSQL_PROMPT, MSSQL_AGENT_PREFIX, MSSQL_AGENT_FORMAT_INSTRUCTIONS

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string))

In [3]:
# Langchain이 Azure OpenAI에 연결하는데 필요한 ENV 함수를 설정합니다
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"] = "azure"

# Install MS SQL DB driver in your machine

SQL DB와 대화를 하기 위해서는 이 컴퓨터에 드라이버가 설치되어 있어야 합니다. 아래 셀을 한번 실행해주세요.
Reference: https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver16&tabs=ubuntu18-install%2Calpine17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline

In [4]:
# !sudo ./download_odbc_driver.sh

# Load Azure SQL DB with the Covid Tracking CSV Data

현재 Azure SQL Database가 비어 있어서 데이터로 채워야 합니다. 이전 노트북에서 사용했던 Covid CSV 파일과 동일한 데이터를 사용하여 결과 및 방법을 비교해 보겠습니다.
이를 위해서는, 생성 시 사용한 자격 증명을 아래 셀에 입력해야 합니다.

In [5]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

db_config = {
                'drivername': 'mssql+pyodbc',
                'username': os.environ["SQL_SERVER_USERNAME"] +'@'+ os.environ["SQL_SERVER_NAME"],
                'password': os.environ["SQL_SERVER_PASSWORD"],
                'host': os.environ["SQL_SERVER_NAME"],
                'port': 1433,
                'database': os.environ["SQL_SERVER_DATABASE"],
                'query': {'driver': 'ODBC Driver 17 for SQL Server'}
            }

# Database에 연결하기 위한 URL object 생성
db_url = URL.create(**db_config)

# Print the resulting URL string
# print(db_url)

# URL String을 사용하여 Azure SQL Database에 연결
engine = create_engine(db_url)

# 연결 상태 테스트
try:
    conn = engine.connect()
    print("Connection successful!")
    result = engine.execute("SELECT @@Version")
    for row in result:
        print(row)
    conn.close()
    
except OperationalError:
    print("Connection failed.")

Connection successful!
('Microsoft SQL Azure (RTM) - 12.0.2000.8 \n\tJul  8 2023 12:00:47 \n\tCopyright (C) 2022 Microsoft Corporation\n',)


/tmp/ipykernel_902797/2845560204.py:27: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  result = engine.execute("SELECT @@Version")


In [7]:
# Pandas Dataframe 형식으로 CSV File을 읽어옵니다
csv_path = "./data/Seoul_Commercial_Districts.csv"
df = pd.read_csv(csv_path).fillna(value = 0)

# 열 이름 및 데이터 유형추론
column_names = df.columns.tolist()
column_types = df.dtypes.to_dict()

# SQL 문을 생성하여 테이블을 만듬
table_name = 'covidtracking'

create_table_sql = f"CREATE TABLE {table_name} ("
for name, dtype in column_types.items():
    if dtype == 'object':
        create_table_sql += f"{name} VARCHAR(MAX), "
    elif dtype == 'int64':
        create_table_sql += f"{name} INT, "
    elif dtype == 'float64':
        create_table_sql += f"{name} FLOAT, "
    elif dtype == 'bool':
        create_table_sql += f"{name} BIT, "
    elif dtype == 'datetime64[ns]':
        create_table_sql += f"{name} DATETIME, "
create_table_sql = create_table_sql[:-2] + ")"

try:
    # Azure SQL에서 table을 만들기
    engine.execute(create_table_sql)
    print("Table",table_name,"succesfully created")
    # SQL Database에 data 삽입
    lower = 0
    upper = 1000
    limit = df.shape[0]

    while lower < limit:
        df[lower:upper].to_sql(table_name, con=engine, if_exists='append', index=False)
        print("rows:", lower, "-", upper, "inserted")
        lower = upper
        upper = min(upper + 1000, limit)

except Exception as e:
    print(e)

Table covidtracking succesfully created
rows: 0 - 1000 inserted
rows: 1000 - 2000 inserted
rows: 2000 - 3000 inserted
rows: 3000 - 4000 inserted
rows: 4000 - 5000 inserted
rows: 5000 - 6000 inserted
rows: 6000 - 7000 inserted
rows: 7000 - 8000 inserted
rows: 8000 - 9000 inserted
rows: 9000 - 10000 inserted
rows: 10000 - 11000 inserted
rows: 11000 - 12000 inserted
rows: 12000 - 13000 inserted
rows: 13000 - 14000 inserted
rows: 14000 - 15000 inserted
rows: 15000 - 16000 inserted
rows: 16000 - 17000 inserted
rows: 17000 - 18000 inserted
rows: 18000 - 19000 inserted
rows: 19000 - 20000 inserted
rows: 20000 - 20780 inserted


# Query with LLM

**Note**: Azure SQL을 사용하고 있지만 Synapse, SQL Managed 인스턴스 또는 다른 SQL 엔진에서도 동일한 코드가 작동합니다. 올바른 ENV 변수만 제공하면 성공적으로 연결됩니다.

In [8]:
# Create or LLM Langchain object using GPT-4 deployment
# GPT-4 배포를 사용하여 Langchain 객체 생성 or LLM
# Agent를 사용할 때 이번에도 GPT-4가 권장됩니다. GPT-35-Turbo는 잘 작동하지 않을 수 있습니다.
llm = AzureChatOpenAI(deployment_name="gpt-4", temperature=0, max_tokens=500)

In [9]:
# db object를 생성
db = SQLDatabase.from_uri(db_url)

In [12]:
# 자연어 질문(쿼리)
QUESTION = "How may patients were hospitalized during July 2020 in Texas, and nationwide as the total of all states? Use the hospitalizedIncrease column"

### SQL Agent

이제 에이전트를 사용하여 React framework가 문제를 어떻게 해결하는지 살펴보도록 하겠습니다.

In [13]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=llm,
    toolkit=toolkit,
    top_k=30,
    verbose=True
)

In [14]:
# Agent는 expert/tools을 사용합니다. 이 SQL Agent의 tool이 무엇인지 살펴보겠습니다.
agent_executor.agent.allowed_tools

['sql_db_query', 'sql_db_schema', 'sql_db_list_tables', 'sql_db_query_checker']

In [15]:
# 프롬프트를 확인해보겠습니다.
printmd(agent_executor.agent.llm_chain.prompt.template)



You are an agent designed to interact with a SQL database.
## Instructions:
- Given an input question, create a syntactically correct mssql query to run, then look at the results of the query and return the answer.
- Unless the user specifies a specific number of examples they wish to obtain, **ALWAYS** limit your query to at most 30 results.
- You can order the results by a relevant column to return the most interesting examples in the database.
- Never query for all the columns from a specific table, only ask for the relevant columns given the question.
- You have access to tools for interacting with the database.
- You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
- DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
- DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE. 
- Your response should be in Markdown. However, **when running  a SQL Query  in "Action Input", do not include the markdown backticks**. Those are only for formatting the response, not for executing the command.
- ALWAYS, as part of your final answer, explain how you got to the answer on a section that starts with: "Explanation:". Include the SQL query as part of the explanation section.
- If the question does not seem related to the database, just return "I don't know" as the answer.
- Only use the below tools. Only use the information returned by the below tools to construct your final answer.

## Tools:



sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', using sql_db_schema to query the correct table fields.
sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: 'table1, table2, table3'
sql_db_list_tables: Input is an empty string, output is a comma separated list of tables in the database.
sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!



## Use the following format:

Question: the input question you must answer. 
Thought: you should always think about what to do. 
Action: the action to take, should be one of [sql_db_query, sql_db_schema, sql_db_list_tables, sql_db_query_checker]. 
Action Input: the input to the action. 
Observation: the result of the action. 
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer. 
Final Answer: the final answer to the original input question. 

Example of Final Answer:
<=== Beginning of example

Action: query_sql_db
Action Input: SELECT TOP (10) [death] FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'
Observation: [(27437.0,), (27088.0,), (26762.0,), (26521.0,), (26472.0,), (26421.0,), (26408.0,)]
Thought:I now know the final answer
Final Answer: There were 27437 people who died of covid in Texas in 2020.

Explanation:
I queried the `covidtracking` table for the `death` column where the state is 'TX' and the date starts with '2020'. The query returned a list of tuples with the number of deaths for each day in 2020. To answer the question, I took the sum of all the deaths in the list, which is 27437. 
I used the following query

```sql
SELECT [death] FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'"
```
===> End of Example



Begin!

Question: {input}
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
{agent_scratchpad}

In [16]:
for i in range(2):
    try:
        response = agent_executor.run(QUESTION) 
        break
    except Exception as e:
        response = str(e)
        continue



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: covidtracking
Thought:The `covidtracking` table seems to be the most relevant one for this question. I should check its schema to understand its structure and the data it contains.
Action: sql_db_schema
Action Input: "covidtracking" 
Observation: 
CREATE TABLE covidtracking (
	date VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	state VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	death FLOAT(53) NULL, 
	[deathConfirmed] FLOAT(53) NULL, 
	[deathIncrease] INTEGER NULL, 
	[deathProbable] FLOAT(53) NULL, 
	hospitalized FLOAT(53) NULL, 
	[hospitalizedCumulative] FLOAT(53) NULL, 
	[hospitalizedCurrently] FLOAT(53) NULL, 
	[hospitalizedIncrease] INTEGER NULL, 
	[inIcuCumulative] FLOAT(53) NULL, 
	[inIcuCurrently] FLOAT(53) NULL, 
	negative FLOAT(53) NULL, 
	[negativeIncrease] INTEGER NULL, 
	[negativeTestsAntibody] FLOAT(53) NULL, 
	[negativeTestsPeopleAntibody] FLOAT(53) NUL

In [17]:
printmd(response)

During July 2020, there were no new hospitalizations in Texas and 63,105 new hospitalizations nationwide.

Explanation:
I queried the `covidtracking` table for the sum of the `hospitalizedIncrease` column where the date starts with '2020-07' and the state is 'TX'. The query returned 0, indicating that there were no new hospitalizations in Texas during July 2020. 

I used the following query:

```sql
SELECT SUM(hospitalizedIncrease) as TotalHospitalized 
FROM covidtracking 
WHERE date LIKE '2020-07%' AND state = 'TX'
```

Then, I queried the `covidtracking` table for the sum of the `hospitalizedIncrease` column where the date starts with '2020-07'. The query returned 63105, indicating that there were 63,105 new hospitalizations nationwide during July 2020.

I used the following query:

```sql
SELECT SUM(hospitalizedIncrease) as TotalHospitalized 
FROM covidtracking 
WHERE date LIKE '2020-07%'
```

**중요**: 질문에 열 이름을 지정하지 않았을 때, 위의 셀을 여러 번 실행하면 다른 결과가 출력되는 경우가 있습니다. <br>
이유는 열 이름이 모호하므로 사람조차도 사용하기에 적합한 열이 무엇인지 식별하기 어렵기 때문입니다.

# Summary

이 노트북에서는 SQL Database에 위치한 데이터셋에 자연어로 질문하는 목표를 달성하였습니다. 우리는 순수하게 프롬프트 엔지니어링(Langchain이 우리에게 제공해주는)과 GPT-4의 인지능력을 활용하여 수행하였습니다.

이 프로세스는 소스에 API와 우리가 인터페이스하기 위해 사용할 수 있는 공통 언어가 있는 한 원래 소스에서 데이터를 이동할 필요가 없는 이유를 보여줍니다. GPT-4는 전체 공개 Github 말뭉치에 대해 훈련되어 있으므로, 존재하는 대부분의 코딩 및 데이터베이스 쿼리 언어를 거의 이해할 수 있습니다.